In [ ]:
!pip install tensorflow
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the data
data = pd.read_csv('network-slicing.csv')

# Select only numerical features for scaling
numerical_features = data.select_dtypes(include=np.number).columns.tolist()
data_numerical = data[numerical_features]

# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_numerical)

# Convert the scaled data back to a DataFrame
data_scaled = pd.DataFrame(data_scaled, columns=numerical_features, index=data.index)

# Rejoin the scaled numerical features with the original categorical features
data_processed = data.copy()
data_processed[numerical_features] = data_scaled[numerical_features]

# ***Encode categorical features using Label Encoding***
categorical_features = data.select_dtypes(exclude=np.number).columns.tolist()
for feature in categorical_features:
    # Convert the column to string type before encoding
    data_processed[feature] = data_processed[feature].astype(str)
    le = LabelEncoder()
    data_processed[feature] = le.fit_transform(data_processed[feature])

# Création des séquences temporelles pour l'entrée LSTM
def create_sequences(data, time_step=10):
    sequences = []
    labels = []
    for i in range(len(data) - time_step):
        sequences.append(data[i:i+time_step])
        labels.append(data[i+time_step])
    return np.array(sequences), np.array(labels)

X, y = create_sequences(data_processed.values, time_step=10)

# Diviser les données en jeu d'entraînement et de test
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape input to be [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

# Créer le model LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=y_train.shape[1]))  # Output layer with the same number of features as y

# Compiler le model
model.compile(loss='mse', optimizer='adam')

# Entraîner le model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Évaluer le model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss}")

# # Calculer l'accuracy  # This may need adjustment depending on your specific problem
# from sklearn.metrics import accuracy_score
# y_pred = model.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1) # Assuming categorical prediction
# y_true_classes = np.argmax(y_test, axis=1)
# accuracy = accuracy_score(y_true_classes, y_pred_

<ipython-input-8-560f72080789>:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('network-slicing.csv')
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 144s 12ms/step - loss: 0.5970
Epoch 2/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 139s 12ms/step - loss: 0.0072
Epoch 3/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 139s 12ms/step - loss: 0.0063
Epoch 4/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 139s 12ms/step - loss: 0.0054
Epoch 5/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 140s 12ms/step - loss: 0.0055
Epoch 6/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 142s 12ms/step - loss: 0.0057
Epoch 7/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 140s 12ms/step - loss: 0.0047
Epoch 8/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 141s 12ms/step - loss: 0.0053
Epoch 9/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 144s 12ms/step - loss: 0.0046
Epoch 10/10
11669/11669 ━━━━━━━━━━━━━━━━━━━━ 144s 12ms/step - loss: 0.0044
Loss: 0.11949394643306732


In [ ]:
# # Calculer l'accuracy  # This may need adjustment depending on your specific problem
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1) # Assuming categorical prediction
y_true_classes = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(accuracy)

2918/2918 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step
0.9810168619973003


**prédire l'état futur du réseau avec LSTM**

In [ ]:
# Faire une prédiction avec LSTM
predicted_state = model.predict(X_test)

# Select only the numerical features for inverse scaling
predicted_state_numerical = predicted_state[:, :len(numerical_features)]

# Redimensionner la prédiction pour l'utiliser dans DRL
predicted_state_numerical = scaler.inverse_transform(predicted_state_numerical)  # Dé-normalisation

# Replace the scaled numerical features in the predicted data
predicted_state[:, :len(numerical_features)] = predicted_state_numerical

2918/2918 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step


**Définir l'environement**

In [ ]:
import gym
from gym import spaces
import numpy as np

class NetworkEnv(gym.Env):
    def __init__(self, state_size, action_size):
        super(NetworkEnv, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.current_state = np.zeros(state_size)
        self.action_space = spaces.Discrete(action_size)
        self.observation_space = spaces.Box(low=0, high=1, shape=(state_size,), dtype=np.float32)

    def reset(self):
        self.current_state = np.zeros(self.state_size)
        return self.current_state

    def step(self, action):
        # Appliquer l'action et obtenir la récompense
        next_state = self.current_state  # Simuler la prochaine étape de l'état
        reward = self.compute_reward(next_state)  # Calculer la récompense
        done = False  # Définir si l'épisode est terminé (à ajuster)
        self.current_state = next_state
        return next_state, reward, done, {}

    def compute_reward(self, state):
        # Calculer la récompense en fonction de l'état du réseau
        return -np.sum(state)  # Exemple : plus l'état est élevé, plus la récompense est basse


**AgentDQN**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.model = self.build_model()
        self.gamma = 0.95  # Facteur de discount
        self.epsilon = 1.0  # Exploration vs exploitation
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.choice(range(self.action_size))  # Exploration
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # Exploitation

    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def train(self, batch_size=32):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.max(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Entraînement du modèle DRL
env = NetworkEnv(state_size=10, action_size=4)
agent = DQNAgent(state_size=10, action_size=4)

for e in range(1000):  # Nombre d'épisodes
    state = env.reset()
    state = np.reshape(state, [1, agent.state_size])
    for time in range(500):  # Nombre d'étapes par épisode
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, agent.state_size])
        agent.memorize(state, action, reward, next_state, done)
        agent.train(batch_size=32)
        state = next_state
        if done:
            break


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

KeyboardInterrupt: 

**Evaluation de LSTM**

*   Calcul de RMSE:Root Mean Squared Error

*   Calcul de MAE: Mean Absolute Error



In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Prédictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Calcul des métriques
rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
mae = mean_absolute_error(y_test_actual, predictions)
print(f"RMSE: {rmse}, MAE: {mae}")


NameError: name 'model' is not defined